In [ ]:
from sklearn.datasets import make_blobs
centers = [(-5, -5), (0, 0), (5, 5), (10, 10)]
X, y = make_blobs(n_samples=400, n_features=2, cluster_std=1.0,
                  centers=centers, shuffle=False, random_state=42)

In [ ]:
%pylab inline

In [ ]:
for k in np.unique(y):
    pylab.plot(X[y == k, 0], X[y == k, 1], '.')

# Naive Bayes

$p(x) = \frac{1}{(2 \pi)^k \mid \Sigma \mid} \exp^{- \frac{1}{2} (x - \mu)^T \Sigma^{-1} (x - \mu)}$

asumiendo independencia

$p(x | c) = \frac{1}{\prod_i 2 \pi \sigma^2_{c_i}} \exp^{- \frac{1}{2} \sum_i \frac{(x - \mu_{c_i})^2}{\sigma_{c_i}^2}}$

In [ ]:
X.shape

In [ ]:
mu = np.array([X[y==k].mean(axis=0) for k in np.unique(y)])

In [ ]:
var = np.array([X[y==k].var(axis=0) for k in np.unique(y)])

In [ ]:
var

In [ ]:
def p_naive(x, mu, var):
    ta = np.prod(2 * np.pi * var, axis=1)
    tb = -0.5 * np.sum((x - mu)**2 / var, axis=1)
    return np.exp(tb) / ta

In [ ]:
p_naive(np.array([0.1, 2.3]), mu, var)

$p(c \mid x) = \frac{p(x \mid c) p(c)}{p(x)}$

$f(x) = \textsf{arg max}_{c \in C} p(c \mid x)$

In [ ]:
_, pc = np.unique(y, return_counts=True)

In [ ]:
pc = pc / pc.sum()

In [ ]:
pc

In [ ]:
p_naive(np.array([0.1, 2.3]), mu, var) * pc

In [ ]:
np.argmax(p_naive(np.array([0.1, 2.3]), mu, var) * pc)

$p(x) = \sum_i p(x \mid c_i) p(c_i)$

In [ ]:
pp = p_naive(np.array([0.1, 2.3]), mu, var) * pc

In [ ]:
pp / np.sum(pp)

In [ ]:
class NB(object):
    def fit(self, X, y):
        self.klass, pc = np.unique(y, return_counts=True)
        self.pc = pc / pc.sum()
        self.mu = np.array([X[y==k].mean(axis=0) for k in self.klass])
        self.var = np.array([X[y==k].var(axis=0) for k in self.klass])
        return self
        
    def predict_proba(self, X):
        p_naive = []
        for _mu, _var in zip(self.mu, self.var):
            ta = np.prod(2 * np.pi * _var)
            _ = (X - _mu)**2 / _var
            tb = -0.5 * np.sum(_, axis=1)
            r = np.exp(tb) / ta
            p_naive.append(r)
        p_naive = np.vstack(p_naive).T * self.pc
        return p_naive / np.atleast_2d(p_naive.sum(axis=1)).T 
    
    def predict(self, X):
        pp = self.predict_proba(X)
        return np.argmax(pp, axis=1)

        

In [ ]:
nb = NB().fit(X, y)

In [ ]:
(nb.predict(X) == y).mean()